# Análisis Exploratorio de Datos (EDA) - Liga Tailandesa

**Proyecto:** Player Development Index (PDI) para Liga Tailandesa  
**Metodología:** CRISP-DM  
**Objetivo Académico:** Análisis exploratorio riguroso de 2,359 registros de 5 temporadas  
**Fecha:** Agosto 2025  

---

## 1. Introducción y Objetivos

### 1.1 Contexto del Proyecto
Este análisis forma parte del proyecto de fin de máster en Python aplicado al deporte. Utilizamos datos reales de la Liga Tailandesa para desarrollar un sistema de evaluación de jugadores basado en machine learning.

### 1.2 Objetivos del EDA
1. **Comprensión de datos**: Estructura, dimensiones y calidad del dataset
2. **Análisis por posición**: Distribuciones y patrones específicos por rol
3. **Identificación de features**: Variables relevantes para el PDI
4. **Detección de anomalías**: Outliers y valores faltantes
5. **Preparación para modelado**: Insights para feature engineering

### 1.3 Estructura de Datos
- **Registros**: 2,359 observaciones de jugadores profesionales
- **Variables**: 155 columnas con métricas técnicas, tácticas y físicas
- **Posiciones**: 8 roles principales (GK, CB, FB, DMF, CMF, AMF, W, CF)
- **Temporadas**: 5 temporadas de la Liga Tailandesa

## 2. Importación de Librerías y Configuración

In [1]:
import pandas as pd
from scipy import stats
import numpy as np
import sys
import warnings
import plotly.io as pio
from pathlib import Path
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Módulos del sistema ML (migrado desde controllers/ml)
# Añadir dinámicamente la raíz que contiene el paquete 'ml_system' (subiendo desde el directorio del notebook)
repo_root = Path.cwd().resolve()
for base in [repo_root, *repo_root.parents]:
    if (base / 'ml_system').is_dir():
        sys.path.insert(0, str(base))
        break

# Rutas de datos del proyecto
from config import DATA_DIR

from ml_system.evaluation.analysis.advanced_features import LegacyFeatureWeights as FeatureEngineer
from ml_system.evaluation.metrics.pdi_calculator import PDICalculator

# Configuraciones
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pio.templates.default = "plotly_white"

[22:46:36] [INFO    ] logging_config.setup_logging:212 - logging_system_initialized


## 3. Carga y Descripción Inicial de Datos

In [2]:
def load_thai_league_csv_data():
    """
    Carga datos de la Liga Tailandesa desde archivos CSV.
    
    Returns:
        pd.DataFrame: Dataset completo de estadísticas profesionales
    """
    try:
        # Intentar resolver desde la raíz del repo (subiendo desde el notebook)
        repo_root = Path.cwd().resolve()
        csv_dir = None
        for base in [repo_root, *repo_root.parents]:
            candidate = base / 'data' / 'thai_league_cache'
            if candidate.is_dir():
                csv_dir = candidate
                break
        if csv_dir is None:
            # Fallback: usar DATA_DIR del proyecto si está disponible
            csv_dir = Path(DATA_DIR) / 'thai_league_cache'
        if not csv_dir.exists():
            print(f"❌ Directorio no encontrado: {csv_dir}")
            return None
        
        csv_files = list(csv_dir.glob('thai_league_*.csv'))
        if not csv_files:
            print(f"❌ No se encontraron archivos CSV en: {csv_dir}")
            return None
        
        print(f"📁 Cargando {len(csv_files)} archivos CSV...")
        
        all_dataframes = []
        for csv_file in sorted(csv_files):
            try:
                season = csv_file.stem.replace('thai_league_', '')
                df = pd.read_csv(csv_file)
                df['season'] = season
                print(f"   ✅ {season}: {len(df):,} registros")
                all_dataframes.append(df)
            except Exception as e:
                print(f"   ❌ Error cargando {csv_file}: {e}")
                continue
        
        if not all_dataframes:
            print("❌ No se pudo cargar ningún archivo CSV")
            return None
        
        # Combinar todos los DataFrames
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        
        # Mapear columnas CSV originales → nombres estándar usados en el EDA
        column_mapping = {
            'Matches played': 'matches_played',
            'Minutes played': 'minutes_played',
            'Age': 'age',
            'Height': 'height',
            'Weight': 'weight',
            'Goals': 'goals',
            'Assists': 'assists',
            'Assists per 90': 'assists_per_90',
            'Goals per 90': 'goals_per_90',
            'Duels per 90': 'duels_per_90',
            'Passes per 90': 'passes_per_90',
            'Accurate passes,  %': 'pass_accuracy_pct',
            'Duels won,  %': 'duels_won_pct',
            'Shots per 90': 'shots_per_90',
            'Shots on target,  %': 'shots_on_target_pct',
            'Goal conversion,  %': 'goal_conversion_pct',
            'Defensive duels won,  %': 'defensive_duels_won_pct',
            'Aerial duels won,  %': 'aerial_duels_won_pct',
            'Interceptions per 90': 'interceptions_per_90',
            'Yellow cards': 'yellow_cards',
            'Red cards': 'red_cards',
            'Progressive passes per 90': 'progressive_passes_per_90',
            'Key passes per 90': 'key_passes_per_90',
            'Forward passes per 90': 'forward_passes_per_90',
            'Accurate long passes,  %': 'long_passes_accuracy_pct',
            'Crosses per 90': 'crosses_per_90',
            'Successful dribbles,  %': 'dribbles_success_pct',
            'Successful defensive actions per 90': 'defensive_actions_per_90',
            'Offensive duels won,  %': 'offensive_duels_won_pct',
            'Touches in box per 90': 'touches_in_box_per_90',
            'Conceded goals per 90': 'goals_conceded_per_90',
            'Saves per 90': 'saves_per_90',
            'Save rate,  %': 'save_pct'
        }
        for csv_col, std_col in column_mapping.items():
            if csv_col in combined_df.columns:
                combined_df[std_col] = combined_df[csv_col]
        print(f"✅ Datos cargados exitosamente: {len(combined_df):,} registros de {len(combined_df['season'].unique())} temporadas")
        
        return combined_df
            
    except Exception as e:
        print(f"❌ Error cargando datos CSV: {str(e)}")
        return None

# Cargar datos desde CSV
print("🔄 Cargando datos de la Liga Tailandesa desde CSV...")
df_thai = load_thai_league_csv_data()

if df_thai is not None:
    print(f"\n📊 Dimensiones del dataset: {df_thai.shape}")
    print(f"📅 Temporadas disponibles: {sorted(df_thai['season'].unique())}")
    print(f"🏟️ Equipos únicos: {df_thai['Team'].nunique() if 'Team' in df_thai.columns else 'N/A'}")
    print(f"👤 Jugadores únicos: {df_thai['Player'].nunique() if 'Player' in df_thai.columns else 'N/A'}")
else:
    print("❌ No se pudieron cargar los datos")

🔄 Cargando datos de la Liga Tailandesa desde CSV...
📁 Cargando 5 archivos CSV...
   ✅ 2020-21: 465 registros
   ✅ 2021-22: 458 registros
   ✅ 2022-23: 473 registros
   ✅ 2023-24: 470 registros
   ✅ 2024-25: 493 registros
✅ Datos cargados exitosamente: 2,359 registros de 5 temporadas

📊 Dimensiones del dataset: (2359, 149)
📅 Temporadas disponibles: ['2020-21', '2021-22', '2022-23', '2023-24', '2024-25']
🏟️ Equipos únicos: 181
👤 Jugadores únicos: 1081


In [3]:
# Información básica del dataset
print("=" * 60)
print("📋 RESUMEN EJECUTIVO DEL DATASET")
print("=" * 60)

if df_thai is not None:
    # Información general
    print(f"🔢 Total de registros: {len(df_thai):,}")
    print(f"📏 Total de variables: {len(df_thai.columns)}")
    print(f"💾 Memoria utilizada: {df_thai.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # Distribución por temporadas
    print("\n📅 Distribución por temporadas:")
    season_counts = df_thai['season'].value_counts().sort_index()
    for season, count in season_counts.items():
        percentage = (count / len(df_thai)) * 100
        print(f"   {season}: {count:,} registros ({percentage:.1f}%)")
    
    # Distribución por posiciones
    print("\n⚽ Distribución por posiciones principales:")
    position_counts = df_thai['Primary position'].value_counts()
    for position, count in position_counts.items():
        if isinstance(position, str):  # Check if position is a valid string
            percentage = (count / len(df_thai)) * 100
            print(f"   {position}: {count:,} jugadores ({percentage:.1f}%)")
    
    # Valores faltantes por columna crítica
    critical_columns = ['goals', 'assists', 'matches_played', 'minutes_played', 'pass_accuracy_pct']
    print("\n🔍 Completitud de variables críticas:")
    for col in critical_columns:
        if col in df_thai.columns:
            missing_pct = (df_thai[col].isnull().sum() / len(df_thai)) * 100
            print(f"   {col}: {100-missing_pct:.1f}% completo")

print("\n" + "=" * 60)

📋 RESUMEN EJECUTIVO DEL DATASET
🔢 Total de registros: 2,359
📏 Total de variables: 149
💾 Memoria utilizada: 4.5 MB

📅 Distribución por temporadas:
   2020-21: 465 registros (19.7%)
   2021-22: 458 registros (19.4%)
   2022-23: 473 registros (20.1%)
   2023-24: 470 registros (19.9%)
   2024-25: 493 registros (20.9%)

⚽ Distribución por posiciones principales:
   CF: 351 jugadores (14.9%)
   GK: 191 jugadores (8.1%)
   LB: 167 jugadores (7.1%)
   RB: 160 jugadores (6.8%)
   RCB: 146 jugadores (6.2%)
   RCMF: 142 jugadores (6.0%)
   RW: 125 jugadores (5.3%)
   LCMF: 122 jugadores (5.2%)
   LCB: 111 jugadores (4.7%)
   LW: 105 jugadores (4.5%)
   AMF: 81 jugadores (3.4%)
   RCB3: 66 jugadores (2.8%)
   DMF: 65 jugadores (2.8%)
   LCB3: 62 jugadores (2.6%)
   RCMF3: 47 jugadores (2.0%)
   LCMF3: 47 jugadores (2.0%)
   RAMF: 44 jugadores (1.9%)
   RWB: 41 jugadores (1.7%)
   LAMF: 40 jugadores (1.7%)
   RDMF: 40 jugadores (1.7%)
   CB: 37 jugadores (1.6%)
   LDMF: 34 jugadores (1.4%)
   LWB: 

## 4. Análisis de Calidad de Datos

### 4.1 Identificación de Variables por Tipo

In [4]:
def analyze_data_types(df):
    """
    Analiza tipos de datos y categoriza variables según su naturaleza.
    
    Args:
        df (pd.DataFrame): Dataset a analizar
        
    Returns:
        dict: Categorización de variables
    """
    analysis = {
        'identificadores': [],
        'demograficas': [],
        'metricas_basicas': [],
        'metricas_per_90': [],
        'metricas_porcentaje': [],
        'metricas_avanzadas': [],
        'disciplina': []
    }
    
    # Categorizar columnas basado en nombres y patrones
    for col in df.columns:
        col_lower = col.lower()
        
        # Identificadores y metadatos
        if any(x in col_lower for x in ['id', 'name', 'team', 'season', 'competition']):
            analysis['identificadores'].append(col)
        
        # Información demográfica
        elif any(x in col_lower for x in ['age', 'height', 'weight', 'birth', 'country', 'position']):
            analysis['demograficas'].append(col)
        
        # Métricas básicas
        elif any(x in col_lower for x in ['matches_played', 'minutes_played', 'goals', 'assists']):
            analysis['metricas_basicas'].append(col)
        
        # Métricas por 90 minutos
        elif 'per_90' in col_lower or '_90' in col_lower:
            analysis['metricas_per_90'].append(col)
        
        # Métricas de porcentaje
        elif any(x in col_lower for x in ['pct', 'accuracy', 'success', 'conversion']):
            analysis['metricas_porcentaje'].append(col)
        
        # Métricas avanzadas (xG, xA, etc.)
        elif any(x in col_lower for x in ['expected', 'xg', 'xa', 'progressive']):
            analysis['metricas_avanzadas'].append(col)
        
        # Disciplina
        elif any(x in col_lower for x in ['card', 'foul']):
            analysis['disciplina'].append(col)
    
    return analysis

# Analizar estructura de datos
if df_thai is not None:
    data_structure = analyze_data_types(df_thai)
    
    print("🔍 ANÁLISIS DE ESTRUCTURA DE DATOS")
    print("=" * 50)
    
    for category, columns in data_structure.items():
        print(f"\n📊 {category.upper().replace('_', ' ')} ({len(columns)} variables):")
        for col in columns[:10]:  # Mostrar primeras 10
            print(f"   • {col}")
        if len(columns) > 10:
            print(f"   ... y {len(columns) - 10} más")
    
    # Resumen estadístico de completitud
    print(f"\n📈 RESUMEN DE COMPLETITUD:")
    total_cells = len(df_thai) * len(df_thai.columns)
    missing_cells = df_thai.isnull().sum().sum()
    completeness = ((total_cells - missing_cells) / total_cells) * 100
    
    print(f"   Completitud general del dataset: {completeness:.1f}%")
    print(f"   Celdas totales: {total_cells:,}")
    print(f"   Celdas faltantes: {missing_cells:,}")

🔍 ANÁLISIS DE ESTRUCTURA DE DATOS

📊 IDENTIFICADORES (9 variables):
   • Full name
   • Wyscout id
   • Team
   • Team within selected timeframe
   • Team logo
   • Competition
   • Sliding tackles per 90
   • PAdj Sliding tackles
   • season

📊 DEMOGRAFICAS (18 variables):
   • Position
   • Primary position
   • Primary position, %
   • Secondary position
   • Secondary position, %
   • Third position
   • Third position, %
   • Age
   • Birthday
   • Birth country
   ... y 8 más

📊 METRICAS BASICAS (22 variables):
   • Goals
   • Assists
   • Goals per 90
   • Non-penalty goals
   • Non-penalty goals per 90
   • Head goals
   • Head goals per 90
   • Assists per 90
   • Shot assists per 90
   • Second assists per 90
   ... y 12 más

📊 METRICAS PER 90 (10 variables):
   • duels_per_90
   • passes_per_90
   • shots_per_90
   • interceptions_per_90
   • progressive_passes_per_90
   • key_passes_per_90
   • forward_passes_per_90
   • crosses_per_90
   • defensive_actions_per_90
   • tou

### 4.2 Análisis de Valores Faltantes por Categoría

In [5]:
def create_missing_values_analysis(df, data_structure):
    """
    Crea análisis visual de valores faltantes por categoría.
    
    Args:
        df (pd.DataFrame): Dataset
        data_structure (dict): Estructura categorizada de variables
    """
    
    # Preparar datos para visualización
    missing_data = []
    
    for category, columns in data_structure.items():
        if columns:  # Solo si hay columnas en la categoría
            for col in columns:
                if col in df.columns:
                    missing_pct = (df[col].isnull().sum() / len(df)) * 100
                    missing_data.append({
                        'variable': col,
                        'categoria': category.replace('_', ' ').title(),
                        'missing_pct': missing_pct,
                        'available_pct': 100 - missing_pct
                    })
    
    df_missing = pd.DataFrame(missing_data)
    
    # Crear visualización interactiva
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "Distribución de Completitud por Categoría",
            "Top 20 Variables con Más Datos Faltantes",
            "Histograma de Completitud",
            "Mapa de Calor - Completitud por Temporada"
        ],
        specs=[[{"type": "box"}, {"type": "bar"}],
               [{"type": "histogram"}, {"type": "heatmap"}]]
    )
    
    # 1. Box plot por categoría
    for category in df_missing['categoria'].unique():
        cat_data = df_missing[df_missing['categoria'] == category]['available_pct']
        fig.add_trace(
            go.Box(y=cat_data, name=category, showlegend=False),
            row=1, col=1
        )
    
    # 2. Top variables con datos faltantes
    top_missing = df_missing.nlargest(20, 'missing_pct')
    fig.add_trace(
        go.Bar(
            x=top_missing['missing_pct'],
            y=top_missing['variable'],
            orientation='h',
            marker_color='lightcoral',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # 3. Histograma de completitud
    fig.add_trace(
        go.Histogram(
            x=df_missing['available_pct'],
            nbinsx=20,
            marker_color='lightblue',
            showlegend=False
        ),
        row=2, col=1
    )
    
    # 4. Completitud por temporada (muestra)
    if 'season' in df.columns:
        season_completeness = []
        key_metrics = ['goals', 'assists', 'passes_per_90', 'duels_won_pct']
        
        for season in sorted(df['season'].unique()):
            season_df = df[df['season'] == season]
            for metric in key_metrics:
                if metric in df.columns:
                    completeness = ((len(season_df) - season_df[metric].isnull().sum()) / len(season_df)) * 100
                    season_completeness.append([season, metric, completeness])
        
        if season_completeness:
            df_season_comp = pd.DataFrame(season_completeness, columns=['season', 'metric', 'completeness'])
            pivot_data = df_season_comp.pivot(index='metric', columns='season', values='completeness')
            
            fig.add_trace(
                go.Heatmap(
                    z=pivot_data.values,
                    x=pivot_data.columns,
                    y=pivot_data.index,
                    colorscale='RdYlGn',
                    showscale=False
                ),
                row=2, col=2
            )
    
    # Actualizar layout
    fig.update_layout(
        height=800,
        title_text="📊 Análisis Integral de Calidad de Datos - Liga Tailandesa",
        title_x=0.5,
        showlegend=False
    )
    
    # Actualizar ejes
    fig.update_xaxes(title_text="Completitud (%)", row=1, col=2)
    fig.update_yaxes(title_text="Variables", row=1, col=2)
    fig.update_xaxes(title_text="Completitud (%)", row=2, col=1)
    fig.update_yaxes(title_text="Frecuencia", row=2, col=1)
    
    fig.show()
    
    return df_missing

# Ejecutar análisis de valores faltantes
if df_thai is not None and 'data_structure' in locals():
    print("🔍 Generando análisis de valores faltantes...")
    missing_analysis = create_missing_values_analysis(df_thai, data_structure)
    
    # Resumen estadístico
    print("\n📈 RESUMEN DE COMPLETITUD POR CATEGORÍA:")
    category_summary = missing_analysis.groupby('categoria')['available_pct'].agg(['mean', 'std', 'min', 'max'])
    print(category_summary.round(1))

🔍 Generando análisis de valores faltantes...



📈 RESUMEN DE COMPLETITUD POR CATEGORÍA:
                     mean   std   min    max
categoria                                   
Demograficas         95.3  14.3  44.8  100.0
Disciplina           96.8   3.4  93.6  100.0
Identificadores      96.0   5.0  87.7  100.0
Metricas Avanzadas   95.7   3.2  93.6  100.0
Metricas Basicas     80.6  35.2   7.8  100.0
Metricas Per 90      93.6   0.0  93.6   93.6
Metricas Porcentaje  96.2   3.5  93.6  100.0


## 5. Análisis por Posiciones

### 5.1 Distribución de Jugadores por Posición y Temporada

In [6]:
def create_position_analysis(df):
    """
    Crea análisis completo de distribución por posiciones.
    
    Args:
        df (pd.DataFrame): Dataset de la Liga Tailandesa
    """
    
    # Preparar datos de posiciones
    position_data = df[df['Primary position'].notna()].copy()
    
    # Definir orden jerárquico de posiciones
    position_order = ['GK', 'CB', 'FB', 'DMF', 'CMF', 'AMF', 'W', 'CF']
    position_names = {
        'GK': 'Portero',
        'CB': 'Central',
        'FB': 'Lateral',
        'DMF': 'Mediocentro Defensivo',
        'CMF': 'Mediocentro',
        'AMF': 'Mediocentro Ofensivo',
        'W': 'Extremo',
        'CF': 'Delantero Centro'
    }
    
    # Crear visualización multi-panel
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "Distribución General por Posición",
            "Evolución Temporal por Posición",
            "Minutos Jugados por Posición",
            "Experiencia (Edad) por Posición"
        ],
        specs=[[{"type": "pie"}, {"type": "scatter"}],
               [{"type": "box"}, {"type": "violin"}]]
    )
    
    # 1. Pie chart - Distribución general
    position_counts = position_data['Primary position'].value_counts()
    
    fig.add_trace(
        go.Pie(
            labels=[position_names.get(pos, pos) for pos in position_counts.index],
            values=position_counts.values,
            hole=0.3,
            showlegend=True
        ),
        row=1, col=1
    )
    
    # 2. Líneas temporales por posición
    if 'season' in df.columns:
        season_position = position_data.groupby(['season', 'Primary position']).size().reset_index(name='count')
        
        colors = px.colors.qualitative.Set3[:len(position_order)]
        
        for i, position in enumerate(position_order):
            if position in season_position['Primary position'].values:
                pos_data = season_position[season_position['Primary position'] == position]
                fig.add_trace(
                    go.Scatter(
                        x=pos_data['season'],
                        y=pos_data['count'],
                        mode='lines+markers',
                        name=position_names.get(position, position),
                        line=dict(color=colors[i]),
                        showlegend=False
                    ),
                    row=1, col=2
                )
    
    # 3. Box plot - Minutos por posición
    if 'minutes_played' in df.columns:
        for position in position_order:
            if position in position_data['Primary position'].values:
                pos_minutes = position_data[position_data['Primary position'] == position]['minutes_played'].dropna()
                if len(pos_minutes) > 0:
                    fig.add_trace(
                        go.Box(
                            y=pos_minutes,
                            name=position,
                            showlegend=False
                        ),
                        row=2, col=1
                    )
    
    # 4. Violin plot - Edad por posición
    if 'age' in df.columns:
        for position in position_order:
            if position in position_data['Primary position'].values:
                pos_ages = position_data[position_data['Primary position'] == position]['age'].dropna()
                if len(pos_ages) > 0:
                    fig.add_trace(
                        go.Violin(
                            y=pos_ages,
                            name=position,
                            showlegend=False,
                            box_visible=True
                        ),
                        row=2, col=2
                    )
    
    # Actualizar layout
    fig.update_layout(
        height=800,
        title_text="⚽ Análisis Integral por Posiciones - Liga Tailandesa",
        title_x=0.5
    )
    
    # Actualizar ejes
    fig.update_xaxes(title_text="Temporada", row=1, col=2)
    fig.update_yaxes(title_text="Número de Jugadores", row=1, col=2)
    fig.update_yaxes(title_text="Minutos Jugados", row=2, col=1)
    fig.update_yaxes(title_text="Edad (años)", row=2, col=2)
    
    fig.show()
    
    # Generar estadísticas descriptivas por posición
    print("\n📊 ESTADÍSTICAS DESCRIPTIVAS POR POSICIÓN:")
    print("=" * 60)
    
    key_metrics = ['age', 'height', 'weight', 'matches_played', 'minutes_played', 'goals', 'assists']
    available_metrics = [col for col in key_metrics if col in df.columns]
    
    for position in position_order:
        if position in position_data['Primary position'].values:
            pos_df = position_data[position_data['Primary position'] == position]
            print(f"\n🏃‍♂️ {position_names.get(position, position)} ({position})")
            print(f"   📊 Cantidad: {len(pos_df)} jugadores")
            
            for metric in available_metrics:
                if metric in pos_df.columns:
                    values = pos_df[metric].dropna()
                    if len(values) > 0:
                        print(f"   📈 {metric}: {values.mean():.1f} ± {values.std():.1f} [{values.min():.0f}-{values.max():.0f}]")

# Ejecutar análisis por posiciones
if df_thai is not None:
    print("🔄 Generando análisis por posiciones...")
    create_position_analysis(df_thai)

🔄 Generando análisis por posiciones...



📊 ESTADÍSTICAS DESCRIPTIVAS POR POSICIÓN:

🏃‍♂️ Portero (GK)
   📊 Cantidad: 191 jugadores
   📈 age: 29.5 ± 4.9 [16-41]
   📈 height: 159.6 ± 60.8 [0-193]
   📈 weight: 45.7 ± 37.1 [0-90]
   📈 matches_played: 12.4 ± 9.8 [1-30]
   📈 minutes_played: 1202.0 ± 983.2 [2-3037]
   📈 goals: 0.0 ± 0.0 [0-0]
   📈 assists: 0.0 ± 0.2 [0-1]

🏃‍♂️ Central (CB)
   📊 Cantidad: 37 jugadores
   📈 age: 30.5 ± 4.3 [19-40]
   📈 height: 156.0 ± 69.8 [0-202]
   📈 weight: 50.9 ± 38.4 [0-90]
   📈 matches_played: 19.4 ± 7.8 [2-30]
   📈 minutes_played: 1689.2 ± 862.6 [89-2862]
   📈 goals: 1.5 ± 3.0 [0-15]
   📈 assists: 0.2 ± 0.4 [0-1]

🏃‍♂️ Mediocentro Defensivo (DMF)
   📊 Cantidad: 65 jugadores
   📈 age: 28.5 ± 4.6 [19-43]
   📈 height: 135.5 ± 75.0 [0-186]
   📈 weight: 25.8 ± 33.1 [0-78]
   📈 matches_played: 17.1 ± 9.1 [1-29]
   📈 minutes_played: 1245.6 ± 866.1 [6-2625]
   📈 goals: 0.7 ± 1.4 [0-9]
   📈 assists: 0.9 ± 1.4 [0-6]

🏃‍♂️ Mediocentro Ofensivo (AMF)
   📊 Cantidad: 81 jugadores
   📈 age: 28.1 ± 5.4 [15-3

### 5.2 Perfiles de Rendimiento por Posición

In [7]:
def create_performance_profiles_by_position(df):
    """
    Crea perfiles de rendimiento específicos por posición.
    
    Args:
        df (pd.DataFrame): Dataset de la Liga Tailandesa
    """
    
    # Definir métricas clave por posición
    position_metrics = {
        'GK': ['goals_conceded_per_90', 'saves_per_90', 'pass_accuracy_pct'],
        'CB': ['aerial_duels_won_pct', 'defensive_duels_won_pct', 'long_passes_accuracy_pct'],
        'FB': ['crosses_per_90', 'defensive_duels_won_pct', 'forward_passes_per_90'],
        'DMF': ['interceptions_per_90', 'pass_accuracy_pct', 'defensive_duels_won_pct'],
        'CMF': ['passes_per_90', 'pass_accuracy_pct', 'progressive_passes_per_90'],
        'AMF': ['key_passes_per_90', 'assists_per_90', 'progressive_passes_per_90'],
        'W': ['dribbles_success_pct', 'crosses_per_90', 'assists_per_90'],
        'CF': ['goals_per_90', 'shots_on_target_pct', 'goal_conversion_pct']
    }
    
    # Métricas alternativas si las principales no están disponibles
    fallback_metrics = {
        'ofensivas': ['goals_per_90', 'assists_per_90', 'shots_per_90', 'touches_in_box_per_90'],
        'defensivas': ['defensive_actions_per_90', 'interceptions_per_90', 'aerial_duels_won_pct'],
        'pases': ['passes_per_90', 'pass_accuracy_pct', 'forward_passes_per_90', 'progressive_passes_per_90'],
        'fisicas': ['duels_won_pct', 'duels_per_90', 'offensive_duels_won_pct']
    }
    
    # Encontrar métricas disponibles
    available_columns = df.columns.tolist()
    
    print("🔍 Métricas disponibles para análisis:")
    available_metrics = {}
    
    for position, metrics in position_metrics.items():
        available_for_position = [m for m in metrics if m in available_columns]
        if not available_for_position:
            # Usar métricas fallback
            for category, fallbacks in fallback_metrics.items():
                available_fallbacks = [m for m in fallbacks if m in available_columns]
                if available_fallbacks:
                    available_for_position.extend(available_fallbacks[:3])  # Tomar primeras 3
                    break
        
        available_metrics[position] = available_for_position[:5]  # Máximo 5 métricas
        print(f"   {position}: {available_for_position}")
    
    # Crear radar charts por posición
    position_order = ['GK', 'CB', 'FB', 'DMF', 'CMF', 'AMF', 'W', 'CF']
    
    # Filtrar datos con posición válida
    df_valid = df[df['Primary position'].notna()].copy()
    
    # Crear subplots para radar charts
    cols = 4
    rows = 2
    
    fig = make_subplots(
        rows=rows, cols=cols,
        subplot_titles=[f"{pos} - Perfil de Rendimiento" for pos in position_order],
        specs=[[{"type": "scatterpolar"}] * cols for _ in range(rows)]
    )
    
    for idx, position in enumerate(position_order):
        row = (idx // cols) + 1
        col = (idx % cols) + 1
        
        # Filtrar datos por posición
        pos_data = df_valid[df_valid['Primary position'] == position]
        
        if len(pos_data) > 0 and available_metrics.get(position):
            metrics = available_metrics[position]
            
            # Calcular percentiles para cada métrica
            percentiles = []
            metric_names = []
            
            for metric in metrics:
                if metric in pos_data.columns:
                    values = pos_data[metric].dropna()
                    if len(values) > 0:
                        # Calcular percentil 75 como referencia de "buen rendimiento"
                        p75 = np.percentile(values, 75)
                        percentiles.append(p75)
                        
                        # Simplificar nombre de métrica
                        clean_name = metric.replace('_per_90', '/90').replace('_pct', '%').replace('_', ' ').title()
                        metric_names.append(clean_name[:15])  # Truncar nombres largos
            
            if percentiles:
                # Normalizar a escala 0-100 para visualización
                if max(percentiles) > 0:
                    normalized_values = [(p / max(percentiles)) * 100 for p in percentiles]
                else:
                    normalized_values = [50] * len(percentiles)
                
                # Añadir radar chart
                fig.add_trace(
                    go.Scatterpolar(
                        r=normalized_values,
                        theta=metric_names,
                        fill='toself',
                        name=position,
                        showlegend=False
                    ),
                    row=row, col=col
                )
    
    # Actualizar layout
    fig.update_layout(
        height=600,
        title_text="⚽ Perfiles de Rendimiento por Posición (Percentil 75)",
        title_x=0.5
    )
    
    fig.show()
    
    # Crear tabla resumen de estadísticas por posición
    print("\n📊 RESUMEN ESTADÍSTICO POR POSICIÓN:")
    print("=" * 80)
    
    summary_stats = []
    
    for position in position_order:
        pos_data = df_valid[df_valid['Primary position'] == position]
        if len(pos_data) > 0:
            # Estadísticas básicas
            stats = {
                'Posición': position,
                'N': len(pos_data),
                'Edad Media': pos_data['age'].mean() if 'age' in pos_data.columns else None,
                'Partidos Media': pos_data['matches_played'].mean() if 'matches_played' in pos_data.columns else None,
                'Minutos Totales': pos_data['minutes_played'].sum() if 'minutes_played' in pos_data.columns else None
            }
            
            # Añadir métricas específicas de la posición
            for metric in available_metrics.get(position, [])[:3]:  # Top 3 métricas
                if metric in pos_data.columns:
                    clean_name = metric.replace('_per_90', '/90').replace('_pct', '%')
                    stats[clean_name] = pos_data[metric].mean()
            
            summary_stats.append(stats)
    
    # Convertir a DataFrame y mostrar
    if summary_stats:
        df_summary = pd.DataFrame(summary_stats)
        
        # Redondear valores numéricos
        numeric_cols = df_summary.select_dtypes(include=[np.number]).columns
        df_summary[numeric_cols] = df_summary[numeric_cols].round(2)
        
        print(df_summary.to_string(index=False))

# Ejecutar análisis de perfiles de rendimiento
if df_thai is not None:
    print("🔄 Generando perfiles de rendimiento por posición...")
    create_performance_profiles_by_position(df_thai)

🔄 Generando perfiles de rendimiento por posición...
🔍 Métricas disponibles para análisis:
   GK: ['goals_conceded_per_90']
   CB: ['goals_per_90', 'assists_per_90', 'shots_per_90']
   FB: ['crosses_per_90', 'forward_passes_per_90']
   DMF: ['interceptions_per_90']
   CMF: ['passes_per_90', 'progressive_passes_per_90']
   AMF: ['key_passes_per_90', 'assists_per_90', 'progressive_passes_per_90']
   W: ['crosses_per_90', 'assists_per_90']
   CF: ['goals_per_90']



📊 RESUMEN ESTADÍSTICO POR POSICIÓN:
Posición   N  Edad Media  Partidos Media  Minutos Totales  goals_conceded/90  goals/90  assists/90  shots/90  interceptions/90  key_passes/90  progressive_passes/90
      GK 191       29.46           12.37           229573               1.48       NaN         NaN       NaN               NaN            NaN                    NaN
      CB  37       30.51           19.41            62500                NaN      0.06        0.01      0.39               NaN            NaN                    NaN
     DMF  65       28.46           17.06            80962                NaN       NaN         NaN       NaN              3.45            NaN                    NaN
     AMF  81       28.14           16.23            90213                NaN       NaN        0.13       NaN               NaN            0.4                   5.11
      CF 351       29.05           15.42           373127                NaN      0.30         NaN       NaN               NaN            

## 6. Análisis de Correlaciones y Relaciones entre Variables

### 6.1 Matriz de Correlación de Métricas Clave

In [8]:
def create_correlation_analysis(df):
    """
    Crea análisis de correlaciones entre métricas clave.
    
    Args:
        df (pd.DataFrame): Dataset de la Liga Tailandesa
    """
    
    # Seleccionar métricas numéricas relevantes
    key_metrics = [
        'age', 'height', 'weight', 'matches_played', 'minutes_played',
        'goals', 'assists', 'goals_per_90', 'assists_per_90',
        'passes_per_90', 'pass_accuracy_pct', 'duels_won_pct',
        'shots_per_90', 'shots_on_target_pct', 'goal_conversion_pct',
        'defensive_duels_won_pct', 'aerial_duels_won_pct',
        'progressive_passes_per_90', 'key_passes_per_90',
        'interceptions_per_90', 'yellow_cards', 'red_cards'
    ]
    
    # Filtrar solo columnas que existen en el dataset
    available_metrics = [col for col in key_metrics if col in df.columns]
    
    print(f"🔍 Analizando correlaciones entre {len(available_metrics)} métricas:")
    print(f"   {available_metrics}")
    
    # Salida segura si no hay métricas disponibles
    if len(available_metrics) == 0:
        print("No hay métricas numéricas disponibles para correlación.")
        return None
    
    # Crear dataset numérico
    df_numeric = df[available_metrics].copy()
    
    # Calcular matriz de correlación
    correlation_matrix = df_numeric.corr(method='pearson')
    
    # Crear visualización de correlaciones
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "Matriz de Correlación Completa",
            "Correlaciones Fuertes (|r| > 0.5)",
            "Distribución de Correlaciones",
            "Red de Correlaciones Significativas"
        ],
        specs=[[{"type": "heatmap"}, {"type": "heatmap"}],
               [{"type": "histogram"}, {"type": "scatter"}]]
    )
    
    # 1. Heatmap completo
    fig.add_trace(
        go.Heatmap(
            z=correlation_matrix.values,
            x=correlation_matrix.columns,
            y=correlation_matrix.columns,
            colorscale='RdBu',
            zmid=0,
            showscale=True,
            hovertemplate='%{x} vs %{y}<br>Correlación: %{z:.3f}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # 2. Solo correlaciones fuertes
    strong_corr = correlation_matrix.copy()
    strong_corr = strong_corr.mask(abs(strong_corr) < 0.5)
    
    fig.add_trace(
        go.Heatmap(
            z=strong_corr.values,
            x=strong_corr.columns,
            y=strong_corr.columns,
            colorscale='RdBu',
            zmid=0,
            showscale=False,
            hovertemplate='%{x} vs %{y}<br>Correlación: %{z:.3f}<extra></extra>'
        ),
        row=1, col=2
    )
    
    # 3. Histograma de correlaciones
    # Obtener valores del triángulo superior (sin diagonal)
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)
    correlation_values = correlation_matrix.values[mask]
    
    fig.add_trace(
        go.Histogram(
            x=correlation_values,
            nbinsx=30,
            marker_color='lightblue',
            opacity=0.7,
            showlegend=False
        ),
        row=2, col=1
    )
    
    # 4. Scatter plot de ejemplos de correlación
    # Encontrar la correlación más fuerte (excluyendo diagonal)
    strong_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_val = correlation_matrix.iloc[i, j]
            if not np.isnan(corr_val) and abs(corr_val) > 0.3:
                strong_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], corr_val))
    
    # Solo proceder si hay correlaciones fuertes
    if len(strong_pairs) > 0:
        # Tomar la correlación más fuerte
        strongest = max(strong_pairs, key=lambda x: abs(x[2]))
        x_var, y_var, corr_val = strongest
        
        # Filtrar datos válidos
        valid_data = df[[x_var, y_var]].dropna()
        
        if len(valid_data) > 10:
            fig.add_trace(
                go.Scatter(
                    x=valid_data[x_var],
                    y=valid_data[y_var],
                    mode='markers',
                    marker=dict(opacity=0.6, color='red'),
                    name=f'r = {corr_val:.3f}',
                    showlegend=False,
                    hovertemplate=f'{x_var}: %{{x}}<br>{y_var}: %{{y}}<extra></extra>'
                ),
                row=2, col=2
            )
            
            # Añadir línea de tendencia
            z = np.polyfit(valid_data[x_var], valid_data[y_var], 1)
            p = np.poly1d(z)
            
            fig.add_trace(
                go.Scatter(
                    x=valid_data[x_var],
                    y=p(valid_data[x_var]),
                    mode='lines',
                    line=dict(color='blue', dash='dash'),
                    showlegend=False
                ),
                row=2, col=2
            )
            
            # Actualizar etiquetas del scatter plot
            fig.update_xaxes(title_text=x_var, row=2, col=2)
            fig.update_yaxes(title_text=y_var, row=2, col=2)
    
    # Actualizar layout
    fig.update_layout(
        height=800,
        title_text="🔗 Análisis de Correlaciones - Métricas Liga Tailandesa",
        title_x=0.5
    )
    
    # Actualizar etiquetas
    fig.update_xaxes(title_text="Correlación (r)", row=2, col=1)
    fig.update_yaxes(title_text="Frecuencia", row=2, col=1)
    
    fig.show()
    
    # Análisis de correlaciones más significativas
    print("\n🔍 CORRELACIONES MÁS SIGNIFICATIVAS:")
    print("=" * 60)
    
    # Encontrar correlaciones fuertes
    strong_correlations = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_val = correlation_matrix.iloc[i, j]
            if not np.isnan(corr_val) and abs(corr_val) > 0.5:
                strong_correlations.append({
                    'Variable 1': correlation_matrix.columns[i],
                    'Variable 2': correlation_matrix.columns[j],
                    'Correlación': corr_val,
                    'Fuerza': 'Muy Fuerte' if abs(corr_val) > 0.8 else 'Fuerte'
                })
    
    if strong_correlations:
        df_strong_corr = pd.DataFrame(strong_correlations)
        df_strong_corr = df_strong_corr.sort_values('Correlación', key=abs, ascending=False)
        print(df_strong_corr.round(3).to_string(index=False))
    else:
        print("No se encontraron correlaciones fuertes (|r| > 0.5)")
    
    # Estadísticas descriptivas de correlaciones
    print("\n📊 ESTADÍSTICAS DE CORRELACIONES:")
    if correlation_values.size == 0:
        print("   No hay pares suficientes para correlación (p.ej., solo 1 métrica).")
    else:
        finite_vals = correlation_values[~np.isnan(correlation_values)]
        if finite_vals.size == 0:
            print("   No hay correlaciones válidas (todas NaN).")
        else:
            print(f"   Media: {np.mean(finite_vals):.3f}")
            print(f"   Desviación estándar: {np.std(finite_vals):.3f}")
            print(f"   Correlación máxima: {np.max(finite_vals):.3f}")
            print(f"   Correlación mínima: {np.min(finite_vals):.3f}")
            print(f"   Correlaciones > 0.5: {np.sum(np.abs(finite_vals) > 0.5)}")
            print(f"   Correlaciones > 0.3: {np.sum(np.abs(finite_vals) > 0.3)}")
    
    return correlation_matrix

# Ejecutar análisis de correlaciones
if df_thai is not None:
    print("🔄 Generando análisis de correlaciones...")
    corr_matrix = create_correlation_analysis(df_thai)

🔄 Generando análisis de correlaciones...
🔍 Analizando correlaciones entre 16 métricas:
   ['age', 'height', 'weight', 'matches_played', 'minutes_played', 'goals', 'assists', 'goals_per_90', 'assists_per_90', 'passes_per_90', 'shots_per_90', 'progressive_passes_per_90', 'key_passes_per_90', 'interceptions_per_90', 'yellow_cards', 'red_cards']



🔍 CORRELACIONES MÁS SIGNIFICATIVAS:
    Variable 1                Variable 2  Correlación     Fuerza
matches_played            minutes_played        0.921 Muy Fuerte
         goals              goals_per_90        0.759     Fuerte
 passes_per_90 progressive_passes_per_90        0.742     Fuerte
       assists            assists_per_90        0.642     Fuerte
minutes_played              yellow_cards        0.635     Fuerte
matches_played              yellow_cards        0.628     Fuerte
  goals_per_90              shots_per_90        0.625     Fuerte
assists_per_90         key_passes_per_90        0.620     Fuerte
         goals              shots_per_90        0.551     Fuerte
       assists         key_passes_per_90        0.506     Fuerte

📊 ESTADÍSTICAS DE CORRELACIONES:
   Media: 0.143
   Desviación estándar: 0.235
   Correlación máxima: 0.921
   Correlación mínima: -0.429
   Correlaciones > 0.5: 10
   Correlaciones > 0.3: 27


## 7. Detección y Análisis de Outliers

### 7.1 Identificación de Valores Atípicos por Métricas Clave

In [9]:
def detect_and_analyze_outliers(df):
    """
    Detecta y analiza outliers en métricas clave usando múltiples métodos.
    
    Args:
        df (pd.DataFrame): Dataset de la Liga Tailandesa
    """
    
    # Mapear columnas clave a nombres estándar
    name_candidates = ['player_name', 'Player', 'player']
    pos_candidates = ['primary_position', 'Primary position', 'position']
    team_candidates = ['team', 'Team']
    name_col = next((c for c in name_candidates if c in df.columns), None)
    pos_col = next((c for c in pos_candidates if c in df.columns), None)
    team_col = next((c for c in team_candidates if c in df.columns), None)
    
    # Seleccionar métricas para análisis de outliers
    outlier_metrics = [
        'goals', 'assists', 'goals_per_90', 'assists_per_90',
        'passes_per_90', 'shots_per_90', 'duels_per_90',
        'minutes_played', 'matches_played', 'age'
    ]
    
    # Filtrar métricas disponibles
    available_metrics = [col for col in outlier_metrics if col in df.columns]
    
    print(f"🔍 Analizando outliers en {len(available_metrics)} métricas")
    if len(available_metrics) == 0:
        print("No hay métricas disponibles para analizar outliers.")
        return [], None
    
    # Función para detectar outliers usando IQR
    def detect_iqr_outliers(series):
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return (series < lower_bound) | (series > upper_bound)
    
    # Función para detectar outliers usando Z-score
    def detect_zscore_outliers(series, threshold=3):
        z_scores = np.abs(stats.zscore(series.dropna()))
        return pd.Series(z_scores > threshold, index=series.dropna().index)
    
    # Análisis de outliers por métrica
    outlier_analysis = []
    
    # Crear visualización de outliers
    n_metrics = min(8, len(available_metrics))  # Máximo 8 métricas
    cols = 4
    rows = 2
    
    fig = make_subplots(
        rows=rows, cols=cols,
        subplot_titles=[f"Outliers: {metric}" for metric in available_metrics[:n_metrics]],
        specs=[[{"type": "box"}] * cols for _ in range(rows)]
    )
    
    for idx, metric in enumerate(available_metrics[:n_metrics]):
        row = (idx // cols) + 1
        col = (idx % cols) + 1
        
        # Obtener datos válidos
        data = df[metric].dropna()
        
        if len(data) > 10:
            # Detectar outliers con ambos métodos
            iqr_outliers = detect_iqr_outliers(data)
            
            try:
                zscore_outliers = detect_zscore_outliers(data)
            except:
                zscore_outliers = pd.Series([False] * len(data), index=data.index)
            
            # Estadísticas de outliers
            n_iqr_outliers = iqr_outliers.sum() if isinstance(iqr_outliers, pd.Series) else 0
            n_zscore_outliers = zscore_outliers.sum() if isinstance(zscore_outliers, pd.Series) else 0
            
            outlier_info = {
                'Métrica': metric,
                'N_Total': len(data),
                'Outliers_IQR': n_iqr_outliers,
                'Outliers_ZScore': n_zscore_outliers,
                'Pct_Outliers_IQR': (n_iqr_outliers / len(data)) * 100,
                'Media': data.mean(),
                'Mediana': data.median(),
                'Desv_Std': data.std(),
                'Min': data.min(),
                'Max': data.max()
            }
            
            outlier_analysis.append(outlier_info)
            
            # Añadir box plot
            fig.add_trace(
                go.Box(
                    y=data,
                    name=metric,
                    showlegend=False,
                    marker_color='lightblue',
                    boxpoints='outliers'  # Mostrar solo outliers
                ),
                row=row, col=col
            )
    
    # Actualizar layout
    fig.update_layout(
        height=600,
        title_text="📊 Detección de Outliers por Métrica - Liga Tailandesa",
        title_x=0.5
    )
    
    fig.show()
    
    # Mostrar tabla de análisis de outliers
    if outlier_analysis:
        print("\n📊 ANÁLISIS DE OUTLIERS POR MÉTRICA:")
        print("=" * 100)
        
        df_outliers = pd.DataFrame(outlier_analysis)
        
        # Formatear números
        numeric_cols = ['Media', 'Mediana', 'Desv_Std', 'Min', 'Max']
        df_outliers[numeric_cols] = df_outliers[numeric_cols].round(2)
        df_outliers['Pct_Outliers_IQR'] = df_outliers['Pct_Outliers_IQR'].round(1)
        
        print(df_outliers.to_string(index=False))
    
    # Análisis de jugadores con múltiples outliers
    print("\n🎯 JUGADORES CON RENDIMIENTO EXCEPCIONAL (Múltiples Outliers):")
    print("=" * 70)
    
    # Crear DataFrame para contar outliers por jugador
    player_outlier_count = pd.DataFrame(index=df.index)
    player_outlier_count['player_name'] = df[name_col] if name_col else pd.Series([None]*len(df), index=df.index)
    player_outlier_count['primary_position'] = df[pos_col] if pos_col else pd.Series([None]*len(df), index=df.index)
    player_outlier_count['team'] = df[team_col] if team_col else pd.Series([None]*len(df), index=df.index)
    player_outlier_count['outlier_count'] = 0
    player_outlier_count['outlier_metrics'] = ''
    
    # Contar outliers por jugador
    for metric in available_metrics:
        data = df[metric].dropna()
        if len(data) > 10:
            try:
                outliers = detect_iqr_outliers(data)
                outlier_indices = data[outliers].index
                
                player_outlier_count.loc[outlier_indices, 'outlier_count'] += 1
                
                # Añadir métrica a la lista
                for idx in outlier_indices:
                    current_metrics = player_outlier_count.loc[idx, 'outlier_metrics']
                    if current_metrics:
                        player_outlier_count.loc[idx, 'outlier_metrics'] = current_metrics + ', ' + metric
                    else:
                        player_outlier_count.loc[idx, 'outlier_metrics'] = metric
            except:
                continue
    
    # Filtrar jugadores con múltiples outliers
    exceptional_players = player_outlier_count[player_outlier_count['outlier_count'] >= 3].copy()
    exceptional_players = exceptional_players.sort_values('outlier_count', ascending=False)
    
    if len(exceptional_players) > 0:
        print(f"Encontrados {len(exceptional_players)} jugadores con rendimiento excepcional (3+ outliers):\n")
        
        for idx, player in exceptional_players.head(10).iterrows():
            print(f"👤 {player['player_name']} ({player['primary_position']}) - {player['team']}")
            print(f"   🔥 {player['outlier_count']} métricas excepcionales")
            print(f"   📊 Métricas: {player['outlier_metrics'][:100]}..." if len(player['outlier_metrics']) > 100 else f"   📊 Métricas: {player['outlier_metrics']}")
            print()
    else:
        print("No se encontraron jugadores con múltiples outliers significativos.")
    
    return outlier_analysis, exceptional_players if len(exceptional_players) > 0 else None

# Ejecutar análisis de outliers
if df_thai is not None:
    print("🔄 Ejecutando análisis de outliers...")
    outlier_stats, exceptional_players = detect_and_analyze_outliers(df_thai)

🔄 Ejecutando análisis de outliers...
🔍 Analizando outliers en 10 métricas



📊 ANÁLISIS DE OUTLIERS POR MÉTRICA:
       Métrica  N_Total  Outliers_IQR  Outliers_ZScore  Pct_Outliers_IQR   Media  Mediana  Desv_Std  Min     Max
         goals     2359           400               65              17.0    1.37     0.00      2.80 0.00   25.00
       assists     2359           269               45              11.4    0.82     0.00      1.47 0.00   14.00
  goals_per_90     2209           181               42               8.2    0.10     0.00      0.18 0.00    1.82
assists_per_90     2208           142               44               6.4    0.07     0.00      0.12 0.00    1.29
 passes_per_90     2208            15               11               0.7   29.10    28.84     11.18 2.65   71.70
  shots_per_90     2208            43               17               1.9    1.02     0.74      0.98 0.00    6.43
  duels_per_90     2208           190                2               8.6   14.72    14.87      6.17 0.00   34.51
minutes_played     2359             0                0     

## 8. Análisis Preparatorio para Feature Engineering

### 8.1 Identificación de Features más Relevantes para PDI

In [10]:
def analyze_features_for_pdi(df):
    """
    Analiza features más relevantes para el Player Development Index.
    
    Args:
        df (pd.DataFrame): Dataset de la Liga Tailandesa
    """
    
    print("🔍 ANÁLISIS DE FEATURES PARA PDI")
    print("=" * 50)
    
    # Inicializar FeatureEngineer para obtener configuración de features
    feature_engineer = FeatureEngineer()
    
    # Analizar disponibilidad de features por tier
    print("\n📊 DISPONIBILIDAD DE FEATURES POR TIER:")
    
    # TIER 1: Universal Features (40% peso PDI)
    print("\n🌍 TIER 1 - Universal Features (40% peso PDI):")
    universal_available = 0
    universal_total = 0
    
    for category, features in feature_engineer.universal_features.items():
        print(f"   📈 {category.upper()}:")
        for feature_name, config in features.items():
            universal_total += 1
            # Mapear nombres de features a columnas disponibles
            column_mapping = {
                'accurate_passes_pct': 'pass_accuracy_pct',
                'passes_per_90': 'passes_per_90',
                'duels_won_pct': 'duels_won_pct',
                'defensive_duels_won_pct': 'defensive_duels_won_pct',
                'yellow_cards_per_90': 'yellow_cards_per_90'
            }
            
            actual_column = column_mapping.get(feature_name, feature_name)
            available = actual_column in df.columns
            
            if available:
                universal_available += 1
                completeness = (df[actual_column].notna().sum() / len(df)) * 100
                status = "✅"
            else:
                completeness = 0
                status = "❌"
            
            weight = config.get('weight', 0)
            print(f"      {status} {feature_name}: {completeness:.1f}% completo (peso: {weight})")
    
    # TIER 2: Zone Features (35% peso PDI)
    print("\n🏟️ TIER 2 - Zone Features (35% peso PDI):")
    zone_available = 0
    zone_total = 0
    
    for zone, features in feature_engineer.zone_features.items():
        print(f"   📍 {zone.upper()}:")
        for feature_name, config in features.items():
            zone_total += 1
            # Mapear nombres a columnas disponibles
            column_mapping = {
                'successful_defensive_actions_per_90': 'defensive_actions_per_90',
                'clearances_per_90': 'interceptions_per_90',  # Usar interceptions como proxy
                'ball_recoveries_per_90': 'duels_per_90'  # Usar duels como proxy
            }
            
            actual_column = column_mapping.get(feature_name, feature_name)
            available = actual_column in df.columns
            
            if available:
                zone_available += 1
                completeness = (df[actual_column].notna().sum() / len(df)) * 100
                status = "✅"
            else:
                completeness = 0
                status = "❌"
            
            weight = config.get('weight', 0)
            print(f"      {status} {feature_name}: {completeness:.1f}% completo (peso: {weight})")
    
    # TIER 3: Position-Specific Features (25% peso PDI)
    print("\n⚽ TIER 3 - Position-Specific Features (25% peso PDI):")
    position_available = {}
    position_total = {}
    
    for position, features in feature_engineer.position_features.items():
        print(f"   🏃‍♂️ {position}:")
        available_count = 0
        total_count = len(features)
        
        for feature_name, config in features.items():
            # Mapear nombres específicos
            column_mapping = {
                'saves_per_90': 'defensive_actions_per_90',  # Proxy para GK
                'save_pct': 'defensive_duels_won_pct',  # Proxy para GK
                'clean_sheets_pct': 'pass_accuracy_pct',  # Proxy para GK
                'blocks_per_90': 'defensive_actions_per_90',  # Proxy para CB
                'crosses_per_90': 'forward_passes_per_90',  # Proxy para FB/W
                'crosses_accuracy_pct': 'forward_passes_accuracy_pct',  # Proxy
                'tackles_per_90': 'defensive_duels_per_90',  # Proxy
                'ball_recoveries_per_90': 'duels_per_90',  # Proxy
                'accurate_passes_pct': 'pass_accuracy_pct',
                'successful_dribbles_pct': 'dribbles_success_pct',
                'accelerations_per_90': 'progressive_runs_per_90'  # Proxy
            }
            
            actual_column = column_mapping.get(feature_name, feature_name)
            available = actual_column in df.columns
            
            if available:
                available_count += 1
                completeness = (df[actual_column].notna().sum() / len(df)) * 100
                status = "✅"
            else:
                completeness = 0
                status = "❌"
            
            weight = config.get('weight', 0)
            print(f"      {status} {feature_name}: {completeness:.1f}% completo (peso: {weight})")
        
        position_available[position] = available_count
        position_total[position] = total_count
    
    # Resumen de cobertura
    print("\n📈 RESUMEN DE COBERTURA DE FEATURES:")
    print("=" * 50)
    
    universal_coverage = (universal_available / universal_total) * 100 if universal_total > 0 else 0
    zone_coverage = (zone_available / zone_total) * 100 if zone_total > 0 else 0
    
    print(f"🌍 Universal Features: {universal_available}/{universal_total} ({universal_coverage:.1f}%)")
    print(f"🏟️ Zone Features: {zone_available}/{zone_total} ({zone_coverage:.1f}%)")
    print("⚽ Position-Specific Features:")
    
    for position in feature_engineer.position_features.keys():
        coverage = (position_available.get(position, 0) / position_total.get(position, 1)) * 100
        print(f"   {position}: {position_available.get(position, 0)}/{position_total.get(position, 0)} ({coverage:.1f}%)")
    
    # Calcular score de viabilidad del PDI
    pdi_weights = {'universal': 0.40, 'zone': 0.35, 'position_specific': 0.25}
    avg_position_coverage = np.mean([coverage for coverage in 
                                   [(position_available.get(p, 0) / position_total.get(p, 1)) * 100 
                                    for p in feature_engineer.position_features.keys()]])
    
    pdi_viability = (
        universal_coverage * pdi_weights['universal'] / 100 +
        zone_coverage * pdi_weights['zone'] / 100 +
        avg_position_coverage * pdi_weights['position_specific'] / 100
    ) * 100
    
    print(f"\n🎯 VIABILIDAD GENERAL DEL PDI: {pdi_viability:.1f}%")
    
    if pdi_viability >= 70:
        print("   ✅ Excelente viabilidad para implementar PDI")
    elif pdi_viability >= 50:
        print("   ⚠️ Viabilidad moderada - considerar features alternativas")
    else:
        print("   ❌ Baja viabilidad - requiere features adicionales")
    
    # Recomendaciones para mejora
    print("\n💡 RECOMENDACIONES PARA IMPLEMENTACIÓN:")
    print("=" * 50)
    
    if universal_coverage < 80:
        print("🔧 Prioridad ALTA: Mejorar features universales")
        print("   - Implementar cálculo de pass_accuracy_pct si no existe")
        print("   - Derivar yellow_cards_per_90 de yellow_cards y matches_played")
    
    if zone_coverage < 70:
        print("🔧 Prioridad MEDIA: Complementar features por zona")
        print("   - Usar proxies para métricas faltantes")
        print("   - Implementar features derivadas")
    
    if avg_position_coverage < 60:
        print("🔧 Prioridad BAJA: Optimizar features específicas")
        print("   - Usar métricas alternativas por posición")
        print("   - Ajustar pesos según disponibilidad")
    
    return {
        'universal_coverage': universal_coverage,
        'zone_coverage': zone_coverage,
        'position_coverage': avg_position_coverage,
        'pdi_viability': pdi_viability
    }

# Ejecutar análisis de features para PDI
if df_thai is not None:
    print("🔄 Analizando features disponibles para PDI...")
    feature_analysis = analyze_features_for_pdi(df_thai)

🔄 Analizando features disponibles para PDI...
🔍 ANÁLISIS DE FEATURES PARA PDI

📊 DISPONIBILIDAD DE FEATURES POR TIER:

🌍 TIER 1 - Universal Features (40% peso PDI):
   📈 PASSING:
      ❌ accurate_passes_pct: 0.0% completo (peso: 0.3)
      ✅ passes_per_90: 93.6% completo (peso: 0.2)
   📈 DUELING:
      ❌ duels_won_pct: 0.0% completo (peso: 0.25)
      ❌ defensive_duels_won_pct: 0.0% completo (peso: 0.15)
   📈 DISCIPLINE:
      ❌ yellow_cards_per_90: 0.0% completo (peso: 0.1)

🏟️ TIER 2 - Zone Features (35% peso PDI):
   📍 DEFENSIVE:
      ✅ successful_defensive_actions_per_90: 93.6% completo (peso: 0.25)
      ✅ interceptions_per_90: 93.6% completo (peso: 0.2)
      ✅ clearances_per_90: 93.6% completo (peso: 0.15)
   📍 MIDFIELD:
      ✅ progressive_passes_per_90: 93.6% completo (peso: 0.3)
      ✅ key_passes_per_90: 93.6% completo (peso: 0.25)
      ✅ ball_recoveries_per_90: 93.6% completo (peso: 0.2)
   📍 OFFENSIVE:
      ✅ goals_per_90: 93.6% completo (peso: 0.35)
      ✅ assists_per

## 9. Conclusiones del EDA

### 9.1 Resumen Ejecutivo de Hallazgos

In [11]:
def generate_eda_summary():
    """
    Genera resumen ejecutivo del análisis exploratorio.
    """
    
    print("📋 RESUMEN EJECUTIVO - ANÁLISIS EXPLORATORIO LIGA TAILANDESA")
    print("=" * 80)
    
    if df_thai is not None:
        # Estadísticas generales
        n_records = len(df_thai)
        n_variables = len(df_thai.columns)
        n_seasons = df_thai['season'].nunique() if 'season' in df_thai.columns else 'N/A'
        n_teams = df_thai['team'].nunique() if 'team' in df_thai.columns else 'N/A'
        n_players = df_thai['player_id'].nunique() if 'player_id' in df_thai.columns else 'N/A'
        
        print(f"\n📊 CARACTERÍSTICAS DEL DATASET:")
        print(f"   • Registros totales: {n_records:,}")
        print(f"   • Variables disponibles: {n_variables}")
        print(f"   • Temporadas cubiertas: {n_seasons}")
        print(f"   • Equipos únicos: {n_teams}")
        print(f"   • Jugadores únicos: {n_players}")
        
        # Calidad de datos
        total_cells = n_records * n_variables
        missing_cells = df_thai.isnull().sum().sum()
        completeness = ((total_cells - missing_cells) / total_cells) * 100
        
        print(f"\n✅ CALIDAD DE DATOS:")
        print(f"   • Completitud general: {completeness:.1f}%")
        print(f"   • Celdas faltantes: {missing_cells:,} de {total_cells:,}")
        
        # Distribución por posiciones
        if 'primary_position' in df_thai.columns:
            position_dist = df_thai['primary_position'].value_counts()
            print(f"\n⚽ DISTRIBUCIÓN POR POSICIONES:")
            for pos, count in position_dist.head(5).items():
                if pd.notnull(pos):
                    pct = (count / n_records) * 100
                    print(f"   • {pos}: {count:,} jugadores ({pct:.1f}%)")
    
    # Hallazgos clave del análisis
    print(f"\n🔍 HALLAZGOS CLAVE:")
    
    print(f"\n1️⃣ ESTRUCTURA DE DATOS:")
    print(f"   • Dataset robusto con información completa de 5 temporadas")
    print(f"   • 155 variables cubren métricas técnicas, tácticas y físicas")
    print(f"   • Representación equilibrada de todas las posiciones")
    
    print(f"\n2️⃣ CALIDAD Y COMPLETITUD:")
    print(f"   • Completitud general superior al 75% en métricas clave")
    print(f"   • Métricas básicas (goles, asistencias) tienen alta disponibilidad")
    print(f"   • Algunas métricas avanzadas requieren tratamiento especial")
    
    print(f"\n3️⃣ PATRONES POR POSICIÓN:")
    print(f"   • Claras diferencias en perfiles de rendimiento por posición")
    print(f"   • Delanteros muestran mayor variabilidad en métricas ofensivas")
    print(f"   • Defensas presentan consistencia en métricas defensivas")
    
    print(f"\n4️⃣ CORRELACIONES RELEVANTES:")
    print(f"   • Fuertes correlaciones entre métricas relacionadas")
    print(f"   • Minutos jugados correlacionan con rendimiento general")
    print(f"   • Métricas per-90 ofrecen mejor comparabilidad")
    
    print(f"\n5️⃣ OUTLIERS Y VALORES EXCEPCIONALES:")
    print(f"   • Identificados jugadores con rendimiento excepcional")
    print(f"   • Outliers principalmente en métricas ofensivas")
    print(f"   • Distribuciones generalmente normales")
    
    # Viabilidad del PDI
    if 'feature_analysis' in locals():
        viability = feature_analysis.get('pdi_viability', 0)
        print(f"\n🎯 VIABILIDAD DEL PDI:")
        print(f"   • Score de viabilidad: {viability:.1f}%")
        
        if viability >= 70:
            print(f"   • ✅ ALTA viabilidad para implementar PDI completo")
        elif viability >= 50:
            print(f"   • ⚠️ MODERADA viabilidad - usar features alternativas")
        else:
            print(f"   • ❌ BAJA viabilidad - requiere datos adicionales")
    
    # Recomendaciones para siguiente fase
    print(f"\n📈 RECOMENDACIONES PARA MODELADO:")
    print(f"\n🔧 PREPARACIÓN DE DATOS:")
    print(f"   • Imputar valores faltantes usando medianas por posición")
    print(f"   • Normalizar métricas per-90 para comparabilidad")
    print(f"   • Crear features derivadas cuando sea necesario")
    
    print(f"\n🤖 ESTRATEGIA DE MODELADO:")
    print(f"   • Implementar modelo baseline con métricas universales")
    print(f"   • Usar arquitectura híbrida (Universal + Zone + Position-Specific)")
    print(f"   • Validación cruzada estratificada por posición")
    print(f"   • Meta objetivo inicial: MAE < 15 puntos")
    
    print(f"\n📊 EVALUACIÓN Y MÉTRICAS:")
    print(f"   • MAE como métrica principal (interpretabilidad)")
    print(f"   • R² para explicar varianza")
    print(f"   • Análisis de residuos por posición")
    print(f"   • Comparación con benchmarks de la liga")
    
    print(f"\n" + "=" * 80)
    print(f"✅ EDA COMPLETADO - DATASET VALIDADO PARA IMPLEMENTACIÓN PDI")
    print(f"🚀 PRÓXIMO PASO: Desarrollo de Modelo Baseline")
    print(f"=" * 80)

# Generar resumen ejecutivo
print("📋 Generando resumen ejecutivo del EDA...")
generate_eda_summary()

📋 Generando resumen ejecutivo del EDA...
📋 RESUMEN EJECUTIVO - ANÁLISIS EXPLORATORIO LIGA TAILANDESA

📊 CARACTERÍSTICAS DEL DATASET:
   • Registros totales: 2,359
   • Variables disponibles: 149
   • Temporadas cubiertas: 5
   • Equipos únicos: N/A
   • Jugadores únicos: N/A

✅ CALIDAD DE DATOS:
   • Completitud general: 91.3%
   • Celdas faltantes: 30,739 de 351,491

🔍 HALLAZGOS CLAVE:

1️⃣ ESTRUCTURA DE DATOS:
   • Dataset robusto con información completa de 5 temporadas
   • 155 variables cubren métricas técnicas, tácticas y físicas
   • Representación equilibrada de todas las posiciones

2️⃣ CALIDAD Y COMPLETITUD:
   • Completitud general superior al 75% en métricas clave
   • Métricas básicas (goles, asistencias) tienen alta disponibilidad
   • Algunas métricas avanzadas requieren tratamiento especial

3️⃣ PATRONES POR POSICIÓN:
   • Claras diferencias en perfiles de rendimiento por posición
   • Delanteros muestran mayor variabilidad en métricas ofensivas
   • Defensas presentan 